In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import sklearn

In [ ]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

# Reading the Data

In [ ]:
# importing meta data file containing product related information
# importing libraries

# import gzip
# import shutil

# Path to the meta data zip file. 'sentiment_analysis' is the folder name under 'My Drive'
# path1 = '/content/gdrive/MyDrive/Capstone_Project_Data/meta_Cell_Phones_and_Accessories.json.gz'

# Path to meta data .json file
# path2 = '/content/gdrive/MyDrive/Capstone_Project_Data/meta_Cell_Phones_and_Accessories.json'

# Unzipping the meta data file
# with gzip.open(path1, 'rb') as f_in:
#    with open(path2, 'wb') as f_out:
#        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Reading the unzipped meta data into a Python list. The result will be a list of dictionaries. 
# import json

# Empty list to store the dictonaries
# phonemetadata = []

# Reading the dictionaries in the json file and appending it to the list phonemetadata[]
# with open(path2, 'r') as f:
#    for line in f:
#        phonemetadata.append(json.loads(line))

In [ ]:
# Getting the number of entries in the phonemetadata list
# len(phonemetadata)

In [ ]:
#converting the list phonemetadata into a data frame
# df_meta=pd.DataFrame(phonemetadata)
# df_meta.head()

In [ ]:
# df_meta.to_csv('/content/gdrive/MyDrive/Capstone_Project_Data/meta_data_file.csv')

In [ ]:
df_meta = pd.read_csv('/content/gdrive/MyDrive/Capstone_Project_Data/meta_data_file.csv')
df_meta.head()

,Unnamed: 0,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,0,"['Cell Phones & Accessories', 'Accessories', '...",NaN,[],NaN,Trollies Sing a Long Party VHS,[],[],NaN,NaN,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,NaN,NaN,NaN,6303195164
1,1,"['Cell Phones & Accessories', 'Accessories', '...",NaN,[],NaN,La Pura VHS,[],[],NaN,Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,NaN,NaN,NaN,630524412X
2,2,"['Cell Phones & Accessories', 'Cases, Holsters...",NaN,"['New items to give you a new good mood, welco...",NaN,Leopard Thanksgiving Easter PC Black Case for ...,[],[],NaN,NaN,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,NaN,NaN,NaN,6310200240
3,3,"['Cell Phones & Accessories', 'Cases, Holsters...",NaN,"['A brand-new, unused, unopened, undamaged ite...",NaN,Puppies Faceplate Hard Case Protector for Net1...,[],[],NaN,Live2Pedal,['Protect your phone with style through this A...,"['>#6,368,825 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,NaN,NaN,NaN,6786501710
4,4,"['Cell Phones & Accessories', 'Cases, Holsters...",NaN,"['A brand-new, unused, unopened, undamaged ite...",NaN,White Wolf Faceplate Protector Hard Case for S...,[],[],NaN,wirelessoutletusa,['Protect your phone with style through this A...,"['>#2,672,454 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,NaN,NaN,NaN,6786501273


In [ ]:
#importing file containing customer reviews
df_reviews = pd.read_csv('/content/gdrive/MyDrive/Capstone_Project_Data/Cell_Phones_and_Accessories_5.csv')
df_reviews.head()

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,review_sentiment
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN,POSITIVE
1,5.0,True,A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN,POSITIVE
2,3.0,True,AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN,NEGATIVE
3,2.0,True,A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN,POSITIVE
4,4.0,True,A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN,POSITIVE


## Checking dataframe containing customer reviews related information, i.e. df_reviews

In [ ]:
# Checking the dataframe containing the customer reviews and related information
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128437 entries, 0 to 1128436
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   overall           1128437 non-null  float64
 1   verified          1128437 non-null  bool   
 2   reviewerID        1128437 non-null  object 
 3   asin              1128437 non-null  object 
 4   style             605241 non-null   object 
 5   reviewerName      1128281 non-null  object 
 6   reviewText        1127669 non-null  object 
 7   summary           1127912 non-null  object 
 8   unixReviewTime    1128437 non-null  int64  
 9   vote              92034 non-null    object 
 10  image             27107 non-null    object 
 11  review_sentiment  1128437 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 95.8+ MB


In [ ]:
#checking the percentage of null values present in each coloumn of df_reviews
round(100*(df_reviews.isnull().sum()/len(df_reviews)),2)

overall              0.00
verified             0.00
reviewerID           0.00
asin                 0.00
style               46.36
reviewerName         0.01
reviewText           0.07
summary              0.05
unixReviewTime       0.00
vote                91.84
image               97.60
review_sentiment     0.00
dtype: float64

In [ ]:
#declaring length of actual dataframe df_reviews
len_df1 = len(df_reviews.index)
print(len_df1)

1128437


#### Rows having null "reviewerName","reviewText" and "summary" will be deleted.

In [ ]:
# removing rows with null values of "reviewerName","reviewText" and "summary"   
df_reviews = df_reviews[~pd.isnull(df_reviews['reviewerName'])]
df_reviews = df_reviews[~pd.isnull(df_reviews['reviewText'])]
df_reviews = df_reviews[~pd.isnull(df_reviews['summary'])]

In [ ]:
#checking the percentage of null values present in each coloumn of df_reviews
round(100*(df_reviews.isnull().sum()/len(df_reviews)),2)

overall              0.00
verified             0.00
reviewerID           0.00
asin                 0.00
style               46.37
reviewerName         0.00
reviewText           0.00
summary              0.00
unixReviewTime       0.00
vote                91.84
image               97.61
review_sentiment     0.00
dtype: float64

#### Column "vote" and "image" can be dropped as missing values percentage is more than 90% in both.


In [ ]:
# Dropping columns "vote" and "image" as more than 90% of cells are empty.
df_reviews = df_reviews.drop(['vote','image'], axis=1)

In [ ]:
#checking the percentage of null values present in each coloumn of df_reviews
round(100*(df_reviews.isnull().sum()/len(df_reviews)),2)

overall              0.00
verified             0.00
reviewerID           0.00
asin                 0.00
style               46.37
reviewerName         0.00
reviewText           0.00
summary              0.00
unixReviewTime       0.00
review_sentiment     0.00
dtype: float64

In [ ]:
# Resetting the index values of dataframe df_reviews
df_reviews = df_reviews.reset_index(drop=True)
df_reviews.head()

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,review_sentiment
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,POSITIVE
1,5.0,True,A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,POSITIVE
2,3.0,True,AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NEGATIVE
3,2.0,True,A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,POSITIVE
4,4.0,True,A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,POSITIVE


### Extracting information from "style" column 


In [ ]:
df_reviews['style'] = df_reviews['style'].apply(lambda x: "Not available" if type(x)==float else x)
df_reviews['style'][0:10]

0    {'Color:': ' Bling'}
1           Not available
2           Not available
3           Not available
4           Not available
5           Not available
6           Not available
7           Not available
8           Not available
9           Not available
Name: style, dtype: object

In [ ]:
style_type = []

for i in range(0,len(df_reviews.index)):
  if df_reviews['style'][i]!="Not available":
    string = df_reviews['style'][i]
    find_style = re.findall(r'\w+', string)
    if find_style[0]!='nan':
      style_type.append(find_style[0])
    else:
      style_type.append("Not available")
  else:
    style_type.append("Not available")

print(style_type[0:10])

['Color', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available']


In [ ]:
df_reviews['style_type'] = style_type

In [ ]:
only_cat = []
for i in range(0,len(style_type)):
  if style_type[i]!="Not available":
    if style_type[i] not in only_cat:
      only_cat.append(style_type[i])
print(only_cat)

['Color', 'Package', 'Size', 'Format', 'Model', 'Product', 'Offer', 'Style', 'Edition', 'Pattern', 'Design', 'Length', 'Item', 'Material']


In [ ]:
string = "{'Color:': '3242 Bling'}"
find_style = re.findall(r'[a-zA-Z]+',string)
find_style

['Color', 'Bling']

In [ ]:
style_color = []

for i in range(0,len(df_reviews.index)):
  if df_reviews['style_type'][i]=="Color":
    string=df_reviews['style'][i]
    find_style = re.findall(r'[a-zA-Z]+', string)
    if len(find_style)>1:
      style_color.append(find_style[1].lower())
    else:
      style_color.append("Not available")
  else:
    style_color.append("Not available")
print(style_color[0:10])


['bling', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available']


In [ ]:
df_reviews['style_color'] = style_color

In [ ]:
# Imputing null values in column "style"
df_reviews.loc[pd.isnull(df_reviews['style']), 'style']="Not available"
#checking the percentage of null values present in each coloumn of df_reviews
round(100*(df_reviews.isnull().sum()/len(df_reviews)),2)

overall             0.0
verified            0.0
reviewerID          0.0
asin                0.0
style               0.0
reviewerName        0.0
reviewText          0.0
summary             0.0
unixReviewTime      0.0
review_sentiment    0.0
style_type          0.0
style_color         0.0
dtype: float64

In [ ]:
df_reviews.head()

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,review_sentiment,style_type,style_color
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,POSITIVE,Color,bling
1,5.0,True,A269FLZCB4GIPV,7508492919,Not available,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,POSITIVE,Not available,Not available
2,3.0,True,AB6CHQWHZW4TV,7508492919,Not available,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NEGATIVE,Not available,Not available
3,2.0,True,A1M117A53LEI8,7508492919,Not available,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,POSITIVE,Not available,Not available
4,4.0,True,A272DUT8M88ZS8,7508492919,Not available,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,POSITIVE,Not available,Not available


In [ ]:
df_reviews.shape

(1127041, 12)

### Checking data retention after cleaning the dataframe df_reviews

In [ ]:
#rows retained

round(100*(len(df_reviews.index)/len_df1),2)

99.88

### Converting unixReviewTime into datetime format

In [ ]:
#defining a function to convert unixreviewtime to datetime format
import datetime
def unixreader(value):
  x = datetime.datetime.fromtimestamp(
        int(value)
    ).strftime('%Y-%m-%d')
  return x

In [ ]:
df_reviews['unixReviewTime'] = df_reviews['unixReviewTime'].apply(lambda x: unixreader(x))
df_reviews['unixReviewTime'][0:10]

0    2014-08-04
1    2014-02-12
2    2014-02-08
3    2014-02-04
4    2014-02-03
5    2014-01-27
6    2014-01-23
7    2014-01-17
8    2013-12-27
9    2013-12-16
Name: unixReviewTime, dtype: object

In [ ]:
df_reviews.head()

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,review_sentiment,style_type,style_color
0,5.0,True,A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,2014-08-04,POSITIVE,Color,bling
1,5.0,True,A269FLZCB4GIPV,7508492919,Not available,sarah ponce,When you don't want to spend a whole lot of ca...,1,2014-02-12,POSITIVE,Not available,Not available
2,3.0,True,AB6CHQWHZW4TV,7508492919,Not available,Kai,"so the case came on time, i love the design. I...",Its okay,2014-02-08,NEGATIVE,Not available,Not available
3,2.0,True,A1M117A53LEI8,7508492919,Not available,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,2014-02-04,POSITIVE,Not available,Not available
4,4.0,True,A272DUT8M88ZS8,7508492919,Not available,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,2014-02-03,POSITIVE,Not available,Not available


## Checking dataframe containing metadata of items sold

In [ ]:
df_meta.head(10)

,Unnamed: 0,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,0,"['Cell Phones & Accessories', 'Accessories', '...",NaN,[],NaN,Trollies Sing a Long Party VHS,[],[],NaN,NaN,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,NaN,NaN,NaN,6303195164
1,1,"['Cell Phones & Accessories', 'Accessories', '...",NaN,[],NaN,La Pura VHS,[],[],NaN,Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,NaN,NaN,NaN,630524412X
2,2,"['Cell Phones & Accessories', 'Cases, Holsters...",NaN,"['New items to give you a new good mood, welco...",NaN,Leopard Thanksgiving Easter PC Black Case for ...,[],[],NaN,NaN,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,NaN,NaN,NaN,6310200240
3,3,"['Cell Phones & Accessories', 'Cases, Holsters...",NaN,"['A brand-new, unused, unopened, undamaged ite...",NaN,Puppies Faceplate Hard Case Protector for Net1...,[],[],NaN,Live2Pedal,['Protect your phone with style through this A...,"['>#6,368,825 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,NaN,NaN,NaN,6786501710
4,4,"['Cell Phones & Accessories', 'Cases, Holsters...",NaN,"['A brand-new, unused, unopened, undamaged ite...",NaN,White Wolf Faceplate Protector Hard Case for S...,[],[],NaN,wirelessoutletusa,['Protect your phone with style through this A...,"['>#2,672,454 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,NaN,NaN,NaN,6786501273
5,5,"['Cell Phones & Accessories', 'Cases, Holsters...",NaN,"['A brand-new, unused, unopened, undamaged ite...",NaN,Camo Duck Grass Rubberized Hard Case Phone Fac...,[],[],NaN,wirelessoutletusa,['Protect your phone with style through this A...,"['>#2,997,672 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,NaN,NaN,NaN,6789300104
6,6,"['Cell Phones & Accessories', 'Accessories', '...",NaN,"['A brand-new, unused, unopened, undamaged ite...",NaN,Camoflague Camo Usa Deer Combo Hybrid Hard Cas...,[],[],NaN,Samsung,['Protect your phone with style through this A...,"['>#2,773,172 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,NaN,NaN,NaN,6786501052
7,7,[],NaN,"[""Product Description\nThe best just got bette...",NaN,Motorola H700 Black - Non-Retail Packaging,[],[],NaN,Motorola,[],[],[],{},Cell Phones & Accessories,NaN,NaN,NaN,7030131746
8,8,"['Cell Phones & Accessories', 'Cases, Holsters...",NaN,['Apple Recommends For Wrap your iPhone 4 in u...,NaN,Juicy Couture Durable 3 Layers Crest Case for ...,[],['https://images-na.ssl-images-amazon.com/imag...,NaN,Juicy Couture,"['Lightweight polycarbonate hard shell', 'Two-...","['>#1,599,758 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,NaN,NaN,NaN,7122120015
9,9,"['Cell Phones & Accessories', 'Accessories']",NaN,[],NaN,Full Diamond Black Skull Snap on Design Case H...,[],['https://images-na.ssl-images-amazon.com/imag...,NaN,edealsaving,[],"['>#4,100,584 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,NaN,NaN,NaN,730214074X


In [ ]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590071 entries, 0 to 590070
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    590071 non-null  int64 
 1   category      590071 non-null  object
 2   tech1         6585 non-null    object
 3   description   590071 non-null  object
 4   fit           43 non-null      object
 5   title         590062 non-null  object
 6   also_buy      590071 non-null  object
 7   image         590071 non-null  object
 8   tech2         231 non-null     object
 9   brand         577311 non-null  object
 10  feature       590071 non-null  object
 11  rank          590071 non-null  object
 12  also_view     590071 non-null  object
 13  details       590071 non-null  object
 14  main_cat      586685 non-null  object
 15  similar_item  50931 non-null   object
 16  date          45880 non-null   object
 17  price         127564 non-null  object
 18  asin          590071 non

In [ ]:
#checking the percentage of null values present in each coloumn of df_reviews
round(100*(df_meta.isnull().sum()/len(df_meta)),2)

Unnamed: 0       0.00
category         0.00
tech1           98.88
description      0.00
fit             99.99
title            0.00
also_buy         0.00
image            0.00
tech2           99.96
brand            2.16
feature          0.00
rank             0.00
also_view        0.00
details          0.00
main_cat         0.57
similar_item    91.37
date            92.22
price           78.38
asin             0.00
dtype: float64

Following columns have more than 90% null values: tech1, fit, tech2, similar_item, date
Having no significant information, these columns can be dropped from the dataframe


In [ ]:
df_meta = df_meta.drop(['tech1','fit','tech2','similar_item','date'], axis=1)
round(100*(df_meta.isnull().sum()/len(df_meta)),2)

Unnamed: 0      0.00
category        0.00
description     0.00
title           0.00
also_buy        0.00
image           0.00
brand           2.16
feature         0.00
rank            0.00
also_view       0.00
details         0.00
main_cat        0.57
price          78.38
asin            0.00
dtype: float64

In [ ]:
df_meta['brand'] = df_meta['brand'].fillna("Not available")
df_meta['main_cat'] = df_meta['main_cat'].fillna("Not available")

round(100*(df_meta.isnull().sum()/len(df_meta)),2)

Unnamed: 0      0.00
category        0.00
description     0.00
title           0.00
also_buy        0.00
image           0.00
brand           0.00
feature         0.00
rank            0.00
also_view       0.00
details         0.00
main_cat        0.00
price          78.38
asin            0.00
dtype: float64

In [ ]:
df_meta['price'] = df_meta['price'].apply(lambda x: "0.00" if pd.isna(x)==True else x.lstrip("$"))
df_meta['price']

0         0.00
1         0.00
2         0.00
3         0.00
4         0.00
          ... 
590066    0.00
590067    0.01
590068    0.00
590069    3.99
590070    0.00
Name: price, Length: 590071, dtype: object

In [ ]:
round(100*(df_meta.isnull().sum()/len(df_meta)),2)

Unnamed: 0     0.0
category       0.0
description    0.0
title          0.0
also_buy       0.0
image          0.0
brand          0.0
feature        0.0
rank           0.0
also_view      0.0
details        0.0
main_cat       0.0
price          0.0
asin           0.0
dtype: float64

#### Format of data in column "Category" is not ready for analysis, thus need to extract information from it

In [ ]:
cat_cell = []
for i in range(0,len(df_meta.index)):
  x = df_meta['category'][i].strip("'[]'").split(',')[0].rstrip("'")
  if len(x)>1:
    cat_cell.append(df_meta['category'][i].strip("'[]'").split(',')[0].rstrip("'"))
  else:
    cat_cell.append("Not available")
print(cat_cell[0:10])

['Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Not available', 'Cell Phones & Accessories', 'Cell Phones & Accessories']


In [ ]:
cat_desc = []
for i in range(0,len(df_meta.index)):
  x = df_meta['category'][i].strip("'[]'").split(',')
  if len(x)>1:
    cat_desc.append(df_meta['category'][i].strip("'[]'").split(',')[1].rstrip("'").lstrip("' "))
  else:
    cat_desc.append("Not available")
print(cat_desc[0:10])

['Accessories', 'Accessories', 'Cases', 'Cases', 'Cases', 'Cases', 'Accessories', 'Not available', 'Cases', 'Accessories']


In [ ]:
uni_desc = []

for i in range(0,len(cat_desc)):
  if cat_desc[i] not in uni_desc:
    uni_desc.append(cat_desc[i])
print(uni_desc)

['Accessories', 'Cases', 'Not available', 'Cell Phones', 'SIM Cards & Prepaid Minutes', 'Mobile Broadband', 'Featured Categories']


In [ ]:
# Creating two new column that will contain product category and sub-category information
df_meta['category_cellphone'] = cat_cell
df_meta['subcategory_cellphone'] = cat_desc
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
0,0,"['Cell Phones & Accessories', 'Accessories', '...",[],Trollies Sing a Long Party VHS,[],[],Not available,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,0.00,6303195164,Cell Phones & Accessories,Accessories
1,1,"['Cell Phones & Accessories', 'Accessories', '...",[],La Pura VHS,[],[],Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,0.00,630524412X,Cell Phones & Accessories,Accessories
2,2,"['Cell Phones & Accessories', 'Cases, Holsters...","['New items to give you a new good mood, welco...",Leopard Thanksgiving Easter PC Black Case for ...,[],[],Not available,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,0.00,6310200240,Cell Phones & Accessories,Cases
3,3,"['Cell Phones & Accessories', 'Cases, Holsters...","['A brand-new, unused, unopened, undamaged ite...",Puppies Faceplate Hard Case Protector for Net1...,[],[],Live2Pedal,['Protect your phone with style through this A...,"['>#6,368,825 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501710,Cell Phones & Accessories,Cases
4,4,"['Cell Phones & Accessories', 'Cases, Holsters...","['A brand-new, unused, unopened, undamaged ite...",White Wolf Faceplate Protector Hard Case for S...,[],[],wirelessoutletusa,['Protect your phone with style through this A...,"['>#2,672,454 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501273,Cell Phones & Accessories,Cases


In [ ]:
# checking for duplicate rows based on "asin" values
duplicate = df_meta[df_meta.duplicated("asin", keep ='first')]
duplicate.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
1848,1848,"['Cell Phones & Accessories', 'Cases, Holsters...",['Whether you travel the globe or just around ...,PalmOne V Leather Belt Clip Case,[],[],Palm,"['For Palm V handheld', 'Soft black leather ca...","['>#3,057,757 in Cell Phones & Accessories (Se...",[],{},All Electronics,2.59,B00000J3Q2,Cell Phones & Accessories,Cases
1849,1849,"['Cell Phones & Accessories', 'Cases, Holsters...",[],PalmOne III Deluxe Leather Case,[],['https://images-na.ssl-images-amazon.com/imag...,Palm,"['For Pilot, PalmPilot, and Palm III handhelds...","['>#4,146,771 in Cell Phones & Accessories (Se...",[],{},All Electronics,0.00,B00000J438,Cell Phones & Accessories,Cases
1850,1850,"['Cell Phones & Accessories', 'Accessories', '...","[""You'll get plenty of power from our handy ad...",PalmOne Modem AC Adapter,[],['https://images-na.ssl-images-amazon.com/imag...,Palm,[],"['>#7,117,198 in Cell Phones & Accessories (Se...",[],{},All Electronics,0.00,B00000JHVQ,Cell Phones & Accessories,Accessories
1851,1851,"['Cell Phones & Accessories', 'Accessories', '...","['', ""Specifically designed for iWatch Smart W...",2 Pack Apple 38mm Watch Screen Protector (38mm...,"['B07C6BSHKL', 'B077HRTNLM', 'B075WPCSHK', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,Live2Pedal,['Precise laser cutting Live2Pedal screen prot...,['>#122 in Cell Phones & Accessories (See Top ...,"['B07CXN96BT', 'B077HRTNLM', 'B07GYL7PCR', 'B0...",{},Cell Phones & Accessories,8.99,B00001O39C,Cell Phones & Accessories,Accessories
1852,1852,"['Cell Phones & Accessories', 'Accessories']",['The EarHugger cellular headset features a so...,EarHugger C9000 Cellular Headset with Boom Mic...,[],[],Ear Hugger,"['Hands free for safety and convenience', 'Inc...","['>#7,517,303 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,B00001OPI3,Cell Phones & Accessories,Accessories


In [ ]:
print(len(duplicate))
print(len(df_meta.index))

715
590071


#### There are 715 duplicate rows on the basis of "asin" column. For analysis purpose, we would only be requiring unique "asin" values. Thus, rows with duplicated "asin" values to be dropped for further analysis.

In [ ]:
# creating a new dataframe for metadata, with only unique "asin" values by dropping duplicate rows from df_meta
df_meta = df_meta.drop_duplicates(subset="asin", keep="first")
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
0,0,"['Cell Phones & Accessories', 'Accessories', '...",[],Trollies Sing a Long Party VHS,[],[],Not available,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,0.00,6303195164,Cell Phones & Accessories,Accessories
1,1,"['Cell Phones & Accessories', 'Accessories', '...",[],La Pura VHS,[],[],Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,0.00,630524412X,Cell Phones & Accessories,Accessories
2,2,"['Cell Phones & Accessories', 'Cases, Holsters...","['New items to give you a new good mood, welco...",Leopard Thanksgiving Easter PC Black Case for ...,[],[],Not available,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,0.00,6310200240,Cell Phones & Accessories,Cases
3,3,"['Cell Phones & Accessories', 'Cases, Holsters...","['A brand-new, unused, unopened, undamaged ite...",Puppies Faceplate Hard Case Protector for Net1...,[],[],Live2Pedal,['Protect your phone with style through this A...,"['>#6,368,825 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501710,Cell Phones & Accessories,Cases
4,4,"['Cell Phones & Accessories', 'Cases, Holsters...","['A brand-new, unused, unopened, undamaged ite...",White Wolf Faceplate Protector Hard Case for S...,[],[],wirelessoutletusa,['Protect your phone with style through this A...,"['>#2,672,454 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501273,Cell Phones & Accessories,Cases


In [ ]:
df_meta = df_meta.reset_index(drop=True)
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
0,0,"['Cell Phones & Accessories', 'Accessories', '...",[],Trollies Sing a Long Party VHS,[],[],Not available,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,0.00,6303195164,Cell Phones & Accessories,Accessories
1,1,"['Cell Phones & Accessories', 'Accessories', '...",[],La Pura VHS,[],[],Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,0.00,630524412X,Cell Phones & Accessories,Accessories
2,2,"['Cell Phones & Accessories', 'Cases, Holsters...","['New items to give you a new good mood, welco...",Leopard Thanksgiving Easter PC Black Case for ...,[],[],Not available,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,0.00,6310200240,Cell Phones & Accessories,Cases
3,3,"['Cell Phones & Accessories', 'Cases, Holsters...","['A brand-new, unused, unopened, undamaged ite...",Puppies Faceplate Hard Case Protector for Net1...,[],[],Live2Pedal,['Protect your phone with style through this A...,"['>#6,368,825 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501710,Cell Phones & Accessories,Cases
4,4,"['Cell Phones & Accessories', 'Cases, Holsters...","['A brand-new, unused, unopened, undamaged ite...",White Wolf Faceplate Protector Hard Case for S...,[],[],wirelessoutletusa,['Protect your phone with style through this A...,"['>#2,672,454 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501273,Cell Phones & Accessories,Cases


In [ ]:
#checking shape of new dataframe after dropping duplicate rows
df_meta.shape

(589356, 16)

#### Format of data in column "description" is contained inside a list, thus need to extract information and store it as string.

In [ ]:
df_meta['description'] = df_meta['description'].apply(lambda x: x.strip("'[]'"))
df_meta['description'][0:10]

0                                                     
1                                                     
2    New items to give you a new good mood, welcome...
3    A brand-new, unused, unopened, undamaged item ...
4    A brand-new, unused, unopened, undamaged item ...
5    A brand-new, unused, unopened, undamaged item ...
6    A brand-new, unused, unopened, undamaged item ...
7    "Product Description\nThe best just got better...
8    Apple Recommends For Wrap your iPhone 4 in uni...
9                                                     
Name: description, dtype: object

In [ ]:
new_desc = []
for i in range(0,len(df_meta.index)):
  if len(df_meta['description'][i])!=0:
    string = df_meta['description'][i]
    find_desc = re.findall(r'\w+', string)
    desc_state = " ".join(find_desc)
    new_desc.append(desc_state.lower())
  else:
    new_desc.append("Not available")
print(new_desc[0:10])

['Not available', 'Not available', 'new items to give you a new good mood welcome to you have a happy shopping trip', 'a brand new unused unopened undamaged item in its original packaging where packaging is applicable', 'a brand new unused unopened undamaged item in its original packaging where packaging is applicable', 'a brand new unused unopened undamaged item in its original packaging where packaging is applicable', 'a brand new unused unopened undamaged item in its original packaging where packaging is applicable', 'product description nthe best just got better with proven 5th generation technology and form the motorola h700 bluetooth headset enhances the popular folding microphone design with superior audio performance echo cancellation technology and an approximate 30 reduction in size discriminating consumers will welcome the motorola h700 because it eliminates wires while delivering crisp clear communications with a 10 meter 30 feet connectivity range new levels of comfort and

In [ ]:
# creating new column "new_description" which contain product information as string
df_meta['description'] = new_desc

In [ ]:
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
0,0,"['Cell Phones & Accessories', 'Accessories', '...",Not available,Trollies Sing a Long Party VHS,[],[],Not available,[],"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,0.00,6303195164,Cell Phones & Accessories,Accessories
1,1,"['Cell Phones & Accessories', 'Accessories', '...",Not available,La Pura VHS,[],[],Maribel Guardia,[],"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,0.00,630524412X,Cell Phones & Accessories,Accessories
2,2,"['Cell Phones & Accessories', 'Cases, Holsters...",new items to give you a new good mood welcome ...,Leopard Thanksgiving Easter PC Black Case for ...,[],[],Not available,[],"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,0.00,6310200240,Cell Phones & Accessories,Cases
3,3,"['Cell Phones & Accessories', 'Cases, Holsters...",a brand new unused unopened undamaged item in ...,Puppies Faceplate Hard Case Protector for Net1...,[],[],Live2Pedal,['Protect your phone with style through this A...,"['>#6,368,825 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501710,Cell Phones & Accessories,Cases
4,4,"['Cell Phones & Accessories', 'Cases, Holsters...",a brand new unused unopened undamaged item in ...,White Wolf Faceplate Protector Hard Case for S...,[],[],wirelessoutletusa,['Protect your phone with style through this A...,"['>#2,672,454 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501273,Cell Phones & Accessories,Cases


### Format of data in column "feature" is contained inside a list, thus need to extract information and store it as string.

In [ ]:
df_meta['feature'] = df_meta['feature'].map(lambda x: x.strip("'[]'"))

In [ ]:
refined_feature = []
for i in range(0,len(df_meta.index)):
  if len(df_meta['feature'][i])!=0:
    string = df_meta['feature'][i]
    feature = re.findall(r'\w+', string)
    join_feat = " ".join(feature)
    refined_feature.append(join_feat.lower())
  else:
    refined_feature.append("Not available")
print(refined_feature[0:10])

['Not available', 'Not available', 'Not available', 'protect your phone with style through this attractive protector case and make it tamper resistance delivers ultimate protection from scratches and molds perfectly to device s shape to highlight its beauty reinforced with hard plastic to the sides to ensure the durability of the case and to prolong the life of cellualar device no tools required just snap on for a new look in seconds', 'protect your phone with style through this attractive protector case and make it tamper resistance delivers ultimate protection from scratches and molds perfectly to device s shape to highlight its beauty reinforced with hard plastic to the sides to ensure the durability of the case and to prolong the life of cellualar device precise slots available for the top and side buttons charger port dock connector headset jack and speaker to allow access to all controls and features on the phone no tools required just snap on for a new look in seconds', 'protect

In [ ]:
df_meta['feature'] = refined_feature
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
0,0,"['Cell Phones & Accessories', 'Accessories', '...",Not available,Trollies Sing a Long Party VHS,[],[],Not available,Not available,"557,379 in Movies & TV (",[],{'ASIN: ': '6303195164'},Movies & TV,0.00,6303195164,Cell Phones & Accessories,Accessories
1,1,"['Cell Phones & Accessories', 'Accessories', '...",Not available,La Pura VHS,[],[],Maribel Guardia,Not available,"1,024,449 in Movies & TV (",[],{'ASIN: ': '630524412X'},Movies & TV,0.00,630524412X,Cell Phones & Accessories,Accessories
2,2,"['Cell Phones & Accessories', 'Cases, Holsters...",new items to give you a new good mood welcome ...,Leopard Thanksgiving Easter PC Black Case for ...,[],[],Not available,Not available,"5,315,017 in Books (",[],"{'Publisher:': 'juduiou', 'ASIN:': '6310200240...",Books,0.00,6310200240,Cell Phones & Accessories,Cases
3,3,"['Cell Phones & Accessories', 'Cases, Holsters...",a brand new unused unopened undamaged item in ...,Puppies Faceplate Hard Case Protector for Net1...,[],[],Live2Pedal,protect your phone with style through this att...,"['>#6,368,825 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501710,Cell Phones & Accessories,Cases
4,4,"['Cell Phones & Accessories', 'Cases, Holsters...",a brand new unused unopened undamaged item in ...,White Wolf Faceplate Protector Hard Case for S...,[],[],wirelessoutletusa,protect your phone with style through this att...,"['>#2,672,454 in Cell Phones & Accessories (Se...",[],{},Cell Phones & Accessories,0.00,6786501273,Cell Phones & Accessories,Cases


In [ ]:
df_meta['details'] = df_meta['details'].apply(lambda x: x.strip("'{}' ' "))
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
0,0,"['Cell Phones & Accessories', 'Accessories', '...",Not available,Trollies Sing a Long Party VHS,[],[],Not available,Not available,"557,379 in Movies & TV (",[],ASIN: ': '6303195164,Movies & TV,0.00,6303195164,Cell Phones & Accessories,Accessories
1,1,"['Cell Phones & Accessories', 'Accessories', '...",Not available,La Pura VHS,[],[],Maribel Guardia,Not available,"1,024,449 in Movies & TV (",[],ASIN: ': '630524412X,Movies & TV,0.00,630524412X,Cell Phones & Accessories,Accessories
2,2,"['Cell Phones & Accessories', 'Cases, Holsters...",new items to give you a new good mood welcome ...,Leopard Thanksgiving Easter PC Black Case for ...,[],[],Not available,Not available,"5,315,017 in Books (",[],"Publisher:': 'juduiou', 'ASIN:': '6310200240',...",Books,0.00,6310200240,Cell Phones & Accessories,Cases
3,3,"['Cell Phones & Accessories', 'Cases, Holsters...",a brand new unused unopened undamaged item in ...,Puppies Faceplate Hard Case Protector for Net1...,[],[],Live2Pedal,protect your phone with style through this att...,"['>#6,368,825 in Cell Phones & Accessories (Se...",[],,Cell Phones & Accessories,0.00,6786501710,Cell Phones & Accessories,Cases
4,4,"['Cell Phones & Accessories', 'Cases, Holsters...",a brand new unused unopened undamaged item in ...,White Wolf Faceplate Protector Hard Case for S...,[],[],wirelessoutletusa,protect your phone with style through this att...,"['>#2,672,454 in Cell Phones & Accessories (Se...",[],,Cell Phones & Accessories,0.00,6786501273,Cell Phones & Accessories,Cases


#### Filtering the metadata dataframe df_meta by selecting only subcategory_cellphone as "Cell Phones"

In [ ]:
# creating new metadata detaframe by applying filter on subcategory_cellphone
df_meta = df_meta.loc[df_meta['subcategory_cellphone']=="Cell Phones"]
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
36,36,"['Cell Phones & Accessories', 'Cell Phones', '...",ice cenior senior phone quad band super big ke...,UNLOCKED DUAL SIM SLOT *CENIOR PHONE* QUAD BAN...,[],['https://images-na.ssl-images-amazon.com/imag...,Ice,2g network quad band gsm 850 900 1800 1900 mhz...,"['>#6,356,920 in Cell Phones & Accessories (Se...",[],,Cell Phones & Accessories,0.00,8050110508,Cell Phones & Accessories,Cell Phones
1274,1274,"['Cell Phones & Accessories', 'Cell Phones', '...",standard package 1 x original phone 1 x qualit...,Nokia 3310 Blue Nokia,"['B075FL4H89', 'B00R25GJJW']",['https://images-na.ssl-images-amazon.com/imag...,Nokia,gsm 900 1800 simple and elegant classic model ...,"['>#32,759 in Electronics (See Top 100 in Elec...","['B075FL4H89', 'B075FKZMR2', 'B00TLWTJLO', 'B0...",,All Electronics,0.00,B00005KBGR,Cell Phones & Accessories,Cell Phones
1295,1295,"['Cell Phones & Accessories', 'Cell Phones']",the nokia 5180i is a handset offered by tracfo...,Nokia 5180i TracFone Prepaid Cell Phone with 1...,[],['https://images-na.ssl-images-amazon.com/imag...,Nokia,Not available,[],[],,Cell Phones & Accessories,0.00,B00005S0M4,Cell Phones & Accessories,Cell Phones
1354,1354,"['Cell Phones & Accessories', 'Cell Phones', '...",based on motorola x92 s i90c phone the i95cl o...,Motorola i95cl Phone (Nextel),[],['https://images-na.ssl-images-amazon.com/imag...,Not available,Not available,"['>#3,798,962 in Cell Phones & Accessories (Se...",[],,Cell Phones & Accessories,0.00,B00006J9HH,Cell Phones & Accessories,Cell Phones
1361,1361,"['Cell Phones & Accessories', 'Cell Phones', '...",this is a gsm samsung r225 cellular phone that...,T-Mobile Dual-Band Wireless Phone,[],['https://images-na.ssl-images-amazon.com/imag...,Samsung,Not available,"['>#5,552,033 in Cell Phones & Accessories (Se...",[],,Cell Phones & Accessories,0.00,B00006LIQB,Cell Phones & Accessories,Cell Phones


In [ ]:
# resetting the index of new metadata dataframe
df_meta = df_meta.reset_index(drop=True)
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone
0,36,"['Cell Phones & Accessories', 'Cell Phones', '...",ice cenior senior phone quad band super big ke...,UNLOCKED DUAL SIM SLOT *CENIOR PHONE* QUAD BAN...,[],['https://images-na.ssl-images-amazon.com/imag...,Ice,2g network quad band gsm 850 900 1800 1900 mhz...,"['>#6,356,920 in Cell Phones & Accessories (Se...",[],,Cell Phones & Accessories,0.00,8050110508,Cell Phones & Accessories,Cell Phones
1,1274,"['Cell Phones & Accessories', 'Cell Phones', '...",standard package 1 x original phone 1 x qualit...,Nokia 3310 Blue Nokia,"['B075FL4H89', 'B00R25GJJW']",['https://images-na.ssl-images-amazon.com/imag...,Nokia,gsm 900 1800 simple and elegant classic model ...,"['>#32,759 in Electronics (See Top 100 in Elec...","['B075FL4H89', 'B075FKZMR2', 'B00TLWTJLO', 'B0...",,All Electronics,0.00,B00005KBGR,Cell Phones & Accessories,Cell Phones
2,1295,"['Cell Phones & Accessories', 'Cell Phones']",the nokia 5180i is a handset offered by tracfo...,Nokia 5180i TracFone Prepaid Cell Phone with 1...,[],['https://images-na.ssl-images-amazon.com/imag...,Nokia,Not available,[],[],,Cell Phones & Accessories,0.00,B00005S0M4,Cell Phones & Accessories,Cell Phones
3,1354,"['Cell Phones & Accessories', 'Cell Phones', '...",based on motorola x92 s i90c phone the i95cl o...,Motorola i95cl Phone (Nextel),[],['https://images-na.ssl-images-amazon.com/imag...,Not available,Not available,"['>#3,798,962 in Cell Phones & Accessories (Se...",[],,Cell Phones & Accessories,0.00,B00006J9HH,Cell Phones & Accessories,Cell Phones
4,1361,"['Cell Phones & Accessories', 'Cell Phones', '...",this is a gsm samsung r225 cellular phone that...,T-Mobile Dual-Band Wireless Phone,[],['https://images-na.ssl-images-amazon.com/imag...,Samsung,Not available,"['>#5,552,033 in Cell Phones & Accessories (Se...",[],,Cell Phones & Accessories,0.00,B00006LIQB,Cell Phones & Accessories,Cell Phones


In [ ]:
df_meta.shape

(9789, 16)

## Also Viewed 

In [ ]:
df_meta['also_view'] = df_meta['also_view'].apply(lambda x: x.strip("[]"))
df_meta['also_view'] = df_meta['also_view'].apply(lambda x: x.split(",") if len(x)!=0 else x)
df_meta['also_view']

0                                                        
1       ['B075FL4H89',  'B075FKZMR2',  'B00TLWTJLO',  ...
2                                                        
3                                                        
4                                                        
                              ...                        
9784                                                     
9785                                                     
9786                                                     
9787                                                     
9788                                                     
Name: also_view, Length: 9789, dtype: object

In [ ]:
also_view_asin = []
y = []
c = 0
m = 0
for i in range(0,len(df_meta.index)):
  if len(df_meta['also_view'][i])!=0:
    for j in range(0,len(df_meta['also_view'][i])):
      x = df_meta['also_view'][i][j].strip("'").lstrip(" '")
      y.append(x)
for i in range(0,len(df_meta.index)):
  d = len(df_meta['also_view'][i])
  d=d+m
  also_view_asin.append(y[c:d])
  c = d
  m = d
print(also_view_asin[0:10])

[[], ['B075FL4H89', 'B075FKZMR2', 'B00TLWTJLO', 'B0757B64H2', 'B00851Q9L0', 'B076BVNGZQ', 'B07BTG3T3Y', 'B0172EEAQQ', 'B076BXCRYV', 'B075FLKKR8', 'B07C87CZ8P', 'B000FJ1XWA', 'B000F9GNOS', 'B075FH9Z29', 'B0757BT4C8', 'B07579T83L', 'B07FSD86KD', 'B076BZ18GS', 'B076BYXNZT', 'B00O5EU7TQ', 'B07HH361YQ', 'B077V416QC', 'B07FSCTVCF', 'B01NBHASEK', 'B01M294YJD', 'B009KSA40Y', 'B00TLWTJ82', 'B00O5EUBSI', 'B000K8IAS6', 'B000BYEUXI', 'B07F334RXH', 'B000HHHM3O', 'B000IEBSXG', 'B073SKZYYZ', 'B000FSJW7E', 'B004SBEHWE', 'B00UWT5DM4', 'B07CL3BCV8', 'B002LSHZPM', 'B005AXM3JS', 'B000MXF2IK', 'B00TLWTJHI', 'B00Q1U31VI', 'B01N1Q4OMO', 'B077SGMQ15', 'B0764HC93B', 'B001CBTLME', 'B07JZCFFQR', 'B00L4DXRS4', 'B00OTZUDKO', 'B07FY7HRHF', 'B00UYCER7Q', 'B00LZCJC38', 'B00JN82EFO', 'B006IN9144'], [], [], [], [], [], [], [], []]


In [ ]:
# getting all asin in a list
amazon_id = []
for i in range(0,len(df_meta.index)):
  amazon_id.append(df_meta['asin'][i])
print(amazon_id[0:10])

['8050110508', 'B00005KBGR', 'B00005S0M4', 'B00006J9HH', 'B00006LIQB', 'B00008KGAG', 'B00008NFA3', 'B00009ETU4', 'B00009PGN0', 'B00009R8S6']


In [ ]:
also_view_brand = []
x = []
for i in range(0,len(amazon_id)):
  for j in range(0,len(also_view_asin)):
    if len(also_view_asin[j])!=0:
      for k in range(0,len(also_view_asin[j])):
        if also_view_asin[j][k]==amazon_id[i]:
          x.append(df_meta['brand'][i])
c = 0
m = 0          
for i in range(0,len(df_meta.index)):
  if len(df_meta['also_view'][i])==0:
    also_view_brand.append("Not available")
  else:
    d = len(df_meta['also_view'][i])
    d=d+m
    also_view_brand.append(x[c:d])
    c = d
    m = d
print(also_view_brand[0:10])



['Not available', ['Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'LG', 'Palm Treo 650', 'Palm Treo 650', 'Palm Treo 650', 'Nokia', 'Nokia', 'Nokia', 'Nokia', 'Motorola', 'Motorola', 'Motorola', 'Motorola', 'Motorola', 'Motorola', 'Motorola', 'Motorola', 'Motorola', 'Motorola', 'Motorola'], 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available']


In [ ]:
print(len(also_view_brand))
print(len(df_meta['also_view']))

9789
9789


In [ ]:
df_meta['also_view_brand'] = also_view_brand
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone,also_view_brand
0,36,"['Cell Phones & Accessories', 'Cell Phones', '...",ice cenior senior phone quad band super big ke...,UNLOCKED DUAL SIM SLOT *CENIOR PHONE* QUAD BAN...,[],['https://images-na.ssl-images-amazon.com/imag...,Ice,2g network quad band gsm 850 900 1800 1900 mhz...,"['>#6,356,920 in Cell Phones & Accessories (Se...",,,Cell Phones & Accessories,0.00,8050110508,Cell Phones & Accessories,Cell Phones,Not available
1,1274,"['Cell Phones & Accessories', 'Cell Phones', '...",standard package 1 x original phone 1 x qualit...,Nokia 3310 Blue Nokia,"['B075FL4H89', 'B00R25GJJW']",['https://images-na.ssl-images-amazon.com/imag...,Nokia,gsm 900 1800 simple and elegant classic model ...,"['>#32,759 in Electronics (See Top 100 in Elec...","['B075FL4H89', 'B075FKZMR2', 'B00TLWTJLO', ...",,All Electronics,0.00,B00005KBGR,Cell Phones & Accessories,Cell Phones,"[Nokia, Nokia, Nokia, Nokia, Nokia, Nokia, Nok..."
2,1295,"['Cell Phones & Accessories', 'Cell Phones']",the nokia 5180i is a handset offered by tracfo...,Nokia 5180i TracFone Prepaid Cell Phone with 1...,[],['https://images-na.ssl-images-amazon.com/imag...,Nokia,Not available,[],,,Cell Phones & Accessories,0.00,B00005S0M4,Cell Phones & Accessories,Cell Phones,Not available
3,1354,"['Cell Phones & Accessories', 'Cell Phones', '...",based on motorola x92 s i90c phone the i95cl o...,Motorola i95cl Phone (Nextel),[],['https://images-na.ssl-images-amazon.com/imag...,Not available,Not available,"['>#3,798,962 in Cell Phones & Accessories (Se...",,,Cell Phones & Accessories,0.00,B00006J9HH,Cell Phones & Accessories,Cell Phones,Not available
4,1361,"['Cell Phones & Accessories', 'Cell Phones', '...",this is a gsm samsung r225 cellular phone that...,T-Mobile Dual-Band Wireless Phone,[],['https://images-na.ssl-images-amazon.com/imag...,Samsung,Not available,"['>#5,552,033 in Cell Phones & Accessories (Se...",,,Cell Phones & Accessories,0.00,B00006LIQB,Cell Phones & Accessories,Cell Phones,Not available


In [ ]:
df_meta['also_view_brand'] = df_meta['also_view_brand'].apply(lambda x: " ".join(x) if x!="Not available" else x)

In [ ]:
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone,also_view_brand
0,36,"['Cell Phones & Accessories', 'Cell Phones', '...",ice cenior senior phone quad band super big ke...,UNLOCKED DUAL SIM SLOT *CENIOR PHONE* QUAD BAN...,[],['https://images-na.ssl-images-amazon.com/imag...,Ice,2g network quad band gsm 850 900 1800 1900 mhz...,"['>#6,356,920 in Cell Phones & Accessories (Se...",,,Cell Phones & Accessories,0.00,8050110508,Cell Phones & Accessories,Cell Phones,Not available
1,1274,"['Cell Phones & Accessories', 'Cell Phones', '...",standard package 1 x original phone 1 x qualit...,Nokia 3310 Blue Nokia,"['B075FL4H89', 'B00R25GJJW']",['https://images-na.ssl-images-amazon.com/imag...,Nokia,gsm 900 1800 simple and elegant classic model ...,"['>#32,759 in Electronics (See Top 100 in Elec...","['B075FL4H89', 'B075FKZMR2', 'B00TLWTJLO', ...",,All Electronics,0.00,B00005KBGR,Cell Phones & Accessories,Cell Phones,Nokia Nokia Nokia Nokia Nokia Nokia Nokia Noki...
2,1295,"['Cell Phones & Accessories', 'Cell Phones']",the nokia 5180i is a handset offered by tracfo...,Nokia 5180i TracFone Prepaid Cell Phone with 1...,[],['https://images-na.ssl-images-amazon.com/imag...,Nokia,Not available,[],,,Cell Phones & Accessories,0.00,B00005S0M4,Cell Phones & Accessories,Cell Phones,Not available
3,1354,"['Cell Phones & Accessories', 'Cell Phones', '...",based on motorola x92 s i90c phone the i95cl o...,Motorola i95cl Phone (Nextel),[],['https://images-na.ssl-images-amazon.com/imag...,Not available,Not available,"['>#3,798,962 in Cell Phones & Accessories (Se...",,,Cell Phones & Accessories,0.00,B00006J9HH,Cell Phones & Accessories,Cell Phones,Not available
4,1361,"['Cell Phones & Accessories', 'Cell Phones', '...",this is a gsm samsung r225 cellular phone that...,T-Mobile Dual-Band Wireless Phone,[],['https://images-na.ssl-images-amazon.com/imag...,Samsung,Not available,"['>#5,552,033 in Cell Phones & Accessories (Se...",,,Cell Phones & Accessories,0.00,B00006LIQB,Cell Phones & Accessories,Cell Phones,Not available


In [ ]:
df_meta['rank'] = df_meta['rank'].apply(lambda x: x.strip("'[]'"))
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone,also_view_brand
0,36,"['Cell Phones & Accessories', 'Cell Phones', '...",ice cenior senior phone quad band super big ke...,UNLOCKED DUAL SIM SLOT *CENIOR PHONE* QUAD BAN...,[],['https://images-na.ssl-images-amazon.com/imag...,Ice,2g network quad band gsm 850 900 1800 1900 mhz...,">#6,356,920 in Cell Phones & Accessories (See ...",,,Cell Phones & Accessories,0.00,8050110508,Cell Phones & Accessories,Cell Phones,Not available
1,1274,"['Cell Phones & Accessories', 'Cell Phones', '...",standard package 1 x original phone 1 x qualit...,Nokia 3310 Blue Nokia,"['B075FL4H89', 'B00R25GJJW']",['https://images-na.ssl-images-amazon.com/imag...,Nokia,gsm 900 1800 simple and elegant classic model ...,">#32,759 in Electronics (See Top 100 in Electr...","['B075FL4H89', 'B075FKZMR2', 'B00TLWTJLO', ...",,All Electronics,0.00,B00005KBGR,Cell Phones & Accessories,Cell Phones,Nokia Nokia Nokia Nokia Nokia Nokia Nokia Noki...
2,1295,"['Cell Phones & Accessories', 'Cell Phones']",the nokia 5180i is a handset offered by tracfo...,Nokia 5180i TracFone Prepaid Cell Phone with 1...,[],['https://images-na.ssl-images-amazon.com/imag...,Nokia,Not available,,,,Cell Phones & Accessories,0.00,B00005S0M4,Cell Phones & Accessories,Cell Phones,Not available
3,1354,"['Cell Phones & Accessories', 'Cell Phones', '...",based on motorola x92 s i90c phone the i95cl o...,Motorola i95cl Phone (Nextel),[],['https://images-na.ssl-images-amazon.com/imag...,Not available,Not available,">#3,798,962 in Cell Phones & Accessories (See ...",,,Cell Phones & Accessories,0.00,B00006J9HH,Cell Phones & Accessories,Cell Phones,Not available
4,1361,"['Cell Phones & Accessories', 'Cell Phones', '...",this is a gsm samsung r225 cellular phone that...,T-Mobile Dual-Band Wireless Phone,[],['https://images-na.ssl-images-amazon.com/imag...,Samsung,Not available,">#5,552,033 in Cell Phones & Accessories (See ...",,,Cell Phones & Accessories,0.00,B00006LIQB,Cell Phones & Accessories,Cell Phones,Not available


In [ ]:
rank_in_cat = []
for i in range(0,len(df_meta.index)):
  if len(df_meta['rank'][i])!=0:
    x = df_meta['rank'][i].split(", ")
    string = x[0]
    rank_val = re.findall(r'\d+', string)
    rank_in_cat.append("".join(rank_val))
  else:
    rank_in_cat.append("Not available")
print(rank_in_cat)

['6356920100', '32759100', 'Not available', '3798962100', '5552033100', '6167064100', '2703779100', '7227749100', '4397029100', '4503269100', '3059436100', '1254799100', '2793203100', '7274158100', '5713484100', '2370486100', '3106601100', '6724956100', '2390256100', '1647579100', '4250972100', '2245317100', '1865571100', '6189900100', '7441382100', 'Not available', '1525664100', '7395599100', '3793077100', '2626571100', '6156040100', '2520478100', '5934698100', '7023723100', '3649732100', '1600328', '3090469100', '2714434100', '877179100', '5841528100', '1520466100', '7181713100', '2392971100', '7491781100', '3517527100', '282879100', '1283087100', '6883746100', '6789356100', '7144432100', '3725715100', '2265046100', '3658208100', '7482866100', '1986997100', '7493640100', '7490148100', '7496630100', '2137991100', '7504553100', '4219546100', '233665100', '1409805100', '6764010100', '5351593100', '436771100', '2410930100', '213261100', '1355234100', '1252595', '2022207100', '2638378100'

In [ ]:
rank_in_sub = []
for i in range(0,len(df_meta.index)):
  if len(df_meta['rank'][i])!=0:
    x = df_meta['rank'][i].split(", ")
    if len(x)>1:
      string1 = x[1]
      rank_val1 = re.findall(r'\d+', string)
      rank_in_sub.append("".join(rank_val1))
    else:
      rank_in_sub.append("Not available")
  else:
    rank_in_sub.append("Not available")
print(rank_in_sub)

['880601100', '880601100', 'Not available', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', 'Not available', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', 'Not available', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', '880601100', 'Not available', '880601100', '880601100', '880601100', '880601100', '880601100', '8806011

In [ ]:
rank_cat_type = []
for i in range(0,len(df_meta.index)):
  if len(df_meta['rank'][i])!=0:
    x = df_meta['rank'][i].split(", ")
    string = x[0]
    find_cat = re.findall(r'\w+ \w+ & \w+ ', string)
    if len(find_cat)!=0:
      rank_cat_type.append(find_cat[0].strip())
    else:
      rank_cat_type.append("Other")
  else:
    rank_cat_type.append("Not available")
print(rank_cat_type)

['Cell Phones & Accessories', 'Other', 'Not available', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Not available', 'Other', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Accessories', 'Cell Phones & Acce

In [ ]:
rank_subcat_type = []
for i in range(0,len(df_meta.index)):
  if len(df_meta['rank'][i])!=0:
    x = df_meta['rank'][i].split(", ")
    if len(x)>1:
      string1 = x[1]
      find_sub_cat = re.findall(r'> \w.+' ,string1)
      if len(find_sub_cat)!=0:
        rank_subcat_type.append(find_sub_cat[0].lstrip("> "))
    else:
      rank_subcat_type.append("Not available")
  else:
    rank_subcat_type.append("Not available")
      
print(rank_subcat_type)

['Unlocked Cell Phones', 'Cell Phones & Accessories > Unlocked Cell Phones', 'Not available', 'Carrier Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones', "Carrier Cell Phones'", 'Carrier Cell Phones', 'Carrier Cell Phones', 'Carrier Cell Phones', "Carrier Cell Phones'", 'Unlocked Cell Phones', 'Carrier Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones', 'Carrier Cell Phones', 'Carrier Cell Phones', 'Carrier Cell Phones', "Unlocked Cell Phones'", 'Not available', 'Cell Phones & Accessories > Carrier Cell Phones', 'Unlocked Cell Phones', "Carrier Cell Phones'", 'Carrier Cell Phones', 'Unlocked Cell Phones', 'Unlocked Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones', 'Unlocked Cell Phones', 'Not available', "Carrier Cell Phones'", 'Carrier Cell Phones', 'Carrier Cell Phones', 'Carrier Cell Phones', "Carrier Cell Phones'"

In [ ]:
rank_subcat_type = ["Unlocked Cell Phones" if x=="Cell Phones & Accessories > Unlocked Cell Phones" else x for x in rank_subcat_type]
print(rank_subcat_type[0:10])

['Unlocked Cell Phones', 'Unlocked Cell Phones', 'Not available', 'Carrier Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones', 'Carrier Cell Phones', 'Unlocked Cell Phones']


In [ ]:
print(len(rank_in_cat))
print(len(rank_in_sub))
print(len(rank_cat_type))
print(len(rank_subcat_type))

9789
9789
9789
9789


In [ ]:
df_meta['rank_cat_type'] = rank_cat_type
df_meta['rank_subcat_type'] = rank_subcat_type
df_meta['rank_in_cat'] = rank_in_cat
df_meta['rank_in_sub'] = rank_in_sub
df_meta.head()

,Unnamed: 0,category,description,title,also_buy,image,brand,feature,rank,also_view,details,main_cat,price,asin,category_cellphone,subcategory_cellphone,also_view_brand,rank_cat_type,rank_subcat_type,rank_in_cat,rank_in_sub
0,36,"['Cell Phones & Accessories', 'Cell Phones', '...",ice cenior senior phone quad band super big ke...,UNLOCKED DUAL SIM SLOT *CENIOR PHONE* QUAD BAN...,[],['https://images-na.ssl-images-amazon.com/imag...,Ice,2g network quad band gsm 850 900 1800 1900 mhz...,">#6,356,920 in Cell Phones & Accessories (See ...",,,Cell Phones & Accessories,0.00,8050110508,Cell Phones & Accessories,Cell Phones,Not available,Cell Phones & Accessories,Unlocked Cell Phones,6356920100,880601100
1,1274,"['Cell Phones & Accessories', 'Cell Phones', '...",standard package 1 x original phone 1 x qualit...,Nokia 3310 Blue Nokia,"['B075FL4H89', 'B00R25GJJW']",['https://images-na.ssl-images-amazon.com/imag...,Nokia,gsm 900 1800 simple and elegant classic model ...,">#32,759 in Electronics (See Top 100 in Electr...","['B075FL4H89', 'B075FKZMR2', 'B00TLWTJLO', ...",,All Electronics,0.00,B00005KBGR,Cell Phones & Accessories,Cell Phones,Nokia Nokia Nokia Nokia Nokia Nokia Nokia Noki...,Other,Unlocked Cell Phones,32759100,880601100
2,1295,"['Cell Phones & Accessories', 'Cell Phones']",the nokia 5180i is a handset offered by tracfo...,Nokia 5180i TracFone Prepaid Cell Phone with 1...,[],['https://images-na.ssl-images-amazon.com/imag...,Nokia,Not available,,,,Cell Phones & Accessories,0.00,B00005S0M4,Cell Phones & Accessories,Cell Phones,Not available,Not available,Not available,Not available,Not available
3,1354,"['Cell Phones & Accessories', 'Cell Phones', '...",based on motorola x92 s i90c phone the i95cl o...,Motorola i95cl Phone (Nextel),[],['https://images-na.ssl-images-amazon.com/imag...,Not available,Not available,">#3,798,962 in Cell Phones & Accessories (See ...",,,Cell Phones & Accessories,0.00,B00006J9HH,Cell Phones & Accessories,Cell Phones,Not available,Cell Phones & Accessories,Carrier Cell Phones,3798962100,880601100
4,1361,"['Cell Phones & Accessories', 'Cell Phones', '...",this is a gsm samsung r225 cellular phone that...,T-Mobile Dual-Band Wireless Phone,[],['https://images-na.ssl-images-amazon.com/imag...,Samsung,Not available,">#5,552,033 in Cell Phones & Accessories (See ...",,,Cell Phones & Accessories,0.00,B00006LIQB,Cell Phones & Accessories,Cell Phones,Not available,Cell Phones & Accessories,Carrier Cell Phones,5552033100,880601100


Using following columns from metadata dataframe:
1. asin
2. brand
3. title
3. price
4. category_cellphone
5. subcategory_cellphone
6. description
7. feature
8. also_view_brand
9. rank_cat_type
10. rank_subcat_type
11. rank_in_cat
12. rank_in_sub

In [ ]:
# Creating a new dataframe by selecting above mentioned columns for further analysis
df_meta1 = df_meta[['asin','brand','title','price','category_cellphone','subcategory_cellphone','description','feature','also_view_brand','rank_cat_type','rank_subcat_type','rank_in_cat','rank_in_sub']]
df_meta1.head()

,asin,brand,title,price,category_cellphone,subcategory_cellphone,description,feature,also_view_brand,rank_cat_type,rank_subcat_type,rank_in_cat,rank_in_sub
0,8050110508,Ice,UNLOCKED DUAL SIM SLOT *CENIOR PHONE* QUAD BAN...,0.00,Cell Phones & Accessories,Cell Phones,ice cenior senior phone quad band super big ke...,2g network quad band gsm 850 900 1800 1900 mhz...,Not available,Cell Phones & Accessories,Unlocked Cell Phones,6356920100,880601100
1,B00005KBGR,Nokia,Nokia 3310 Blue Nokia,0.00,Cell Phones & Accessories,Cell Phones,standard package 1 x original phone 1 x qualit...,gsm 900 1800 simple and elegant classic model ...,Nokia Nokia Nokia Nokia Nokia Nokia Nokia Noki...,Other,Unlocked Cell Phones,32759100,880601100
2,B00005S0M4,Nokia,Nokia 5180i TracFone Prepaid Cell Phone with 1...,0.00,Cell Phones & Accessories,Cell Phones,the nokia 5180i is a handset offered by tracfo...,Not available,Not available,Not available,Not available,Not available,Not available
3,B00006J9HH,Not available,Motorola i95cl Phone (Nextel),0.00,Cell Phones & Accessories,Cell Phones,based on motorola x92 s i90c phone the i95cl o...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3798962100,880601100
4,B00006LIQB,Samsung,T-Mobile Dual-Band Wireless Phone,0.00,Cell Phones & Accessories,Cell Phones,this is a gsm samsung r225 cellular phone that...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,5552033100,880601100


In [ ]:
len(df_meta1.index)

9789

In [ ]:
len(df_reviews.index)

1127041

In [ ]:
df_reviews.to_csv('/content/gdrive/MyDrive/Capstone_Project_Data/phone_reviews.csv')
df_meta1.to_csv('/content/gdrive/MyDrive/Capstone_Project_Data/phone_metadata.csv')

In [ ]:
# Merging the two dataframes, phone reviews and meta data to build a common dataframe for EDA
df = pd.merge(df_reviews,df_meta1, how='inner', on='asin')
df.head()

,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,review_sentiment,style_type,style_color,brand,title,price,category_cellphone,subcategory_cellphone,description,feature,also_view_brand,rank_cat_type,rank_subcat_type,rank_in_cat,rank_in_sub
0,1.0,False,A5JLAU2ARJ0BO,B0000E3GWH,Not available,Old-and-Wise,This phone is ugly and heavy and has a terribl...,Bad reception,2004-09-21,NEGATIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100
1,4.0,False,A1ACM1CBGORBN1,B0000E3GWH,Not available,Sherry Berry,I had the Samsung V205 and then I decided to t...,Pretty good phone.. an improvement from my v205,2004-07-24,POSITIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100
2,3.0,False,A2V48Q03FZUOSD,B0000E3GWH,Not available,Benjamin David,This is the first Samsung phone I have had and...,Not as user friendly as the Motorola V300-,2004-07-06,NEGATIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100
3,5.0,True,A1V3TRGWOMA8LC,B0000E3GWH,Not available,James Beswick,"..and I'm from Europe, where the phones and ne...",The best phone I've ever owned,2004-04-12,POSITIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100
4,3.0,False,A3NOBH42C7UI5M,B0000E3GWH,Not available,Kepela,This phone is amazingly small and light for a ...,I love the phone but it has some real problems,2004-02-19,POSITIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100


In [ ]:
len(df.index)

63812

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63812 entries, 0 to 63811
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   overall                63812 non-null  float64
 1   verified               63812 non-null  bool   
 2   reviewerID             63812 non-null  object 
 3   asin                   63812 non-null  object 
 4   style                  63812 non-null  object 
 5   reviewerName           63812 non-null  object 
 6   reviewText             63812 non-null  object 
 7   summary                63812 non-null  object 
 8   unixReviewTime         63812 non-null  object 
 9   review_sentiment       63812 non-null  object 
 10  style_type             63812 non-null  object 
 11  style_color            63812 non-null  object 
 12  brand                  63812 non-null  object 
 13  title                  63812 non-null  object 
 14  price                  63812 non-null  object 
 15  ca

In [ ]:
# df_meta.to_csv('/content/gdrive/MyDrive/Capstone_Project_Data/meta_data_file.csv')
df.to_csv('/content/gdrive/MyDrive/Capstone_Project_Data/merged_data.csv')

In [ ]:
# Importing the merged dataset
merge_df = pd.read_csv('/content/gdrive/MyDrive/Capstone_Project_Data/merged_data.csv')
merge_df.head()

,Unnamed: 0,overall,verified,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,review_sentiment,style_type,style_color,brand,title,price,category_cellphone,subcategory_cellphone,description,feature,also_view_brand,rank_cat_type,rank_subcat_type,rank_in_cat,rank_in_sub
0,0,1.0,False,A5JLAU2ARJ0BO,B0000E3GWH,Not available,Old-and-Wise,This phone is ugly and heavy and has a terribl...,Bad reception,2004-09-21,NEGATIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100
1,1,4.0,False,A1ACM1CBGORBN1,B0000E3GWH,Not available,Sherry Berry,I had the Samsung V205 and then I decided to t...,Pretty good phone.. an improvement from my v205,2004-07-24,POSITIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100
2,2,3.0,False,A2V48Q03FZUOSD,B0000E3GWH,Not available,Benjamin David,This is the first Samsung phone I have had and...,Not as user friendly as the Motorola V300-,2004-07-06,NEGATIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100
3,3,5.0,True,A1V3TRGWOMA8LC,B0000E3GWH,Not available,James Beswick,"..and I'm from Europe, where the phones and ne...",The best phone I've ever owned,2004-04-12,POSITIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100
4,4,3.0,False,A3NOBH42C7UI5M,B0000E3GWH,Not available,Kepela,This phone is amazingly small and light for a ...,I love the phone but it has some real problems,2004-02-19,POSITIVE,Not available,Not available,Samsung,Samsung SGH E715 - Titanium blue (T-Mobile) Ce...,0.00,Cell Phones & Accessories,Cell Phones,compatible with t mobile cellular service sams...,Not available,Not available,Cell Phones & Accessories,Carrier Cell Phones,3059436100,880601100


In [ ]:
merge_df = merge_df.drop("Unnamed: 0", axis=1)
merge_df = merge_df.loc[~pd.isnull(merge_df['reviewText'])]

End of Data Understanding and Preprocessing. We will perform EDA using the merged dataframe created above in a different notebook.